In [ ]:
/// 诊断种类
s BillFlag1=$P($G(^MRC("ID",+ICDRowid)),"^",13)
s BillFlag3=$P($G(^MRC("ID",+ICDRowid)),"^",15)
s DiagObj.DiagnosisClass=$SELECT((BillFlag1="Y")&&(BillFlag3="Y"):"证型",BillFlag3="Y":"中医",1:"西医")


In [ ]:
    if $g(Adm)="" q ""
	s MRAdm=$p($g(^PAADM(Adm)),"^",61)
	q:$g(MRAdm)="" ""
	Set DiagStr="",Diag=0
	For  Set Diag=$O(^MR(MRAdm,"DIA",Diag)) Quit:(Diag="")  Do
	.Set DiagID=$P($g(^MR(MRAdm,"DIA",Diag)),"^",1)
	.s DiagTypeID=$o(^MR(MRAdm,"DIA",Diag,"TYP",0))
	.s TypeID="",TypeDesc=""
	.i DiagTypeID'="" s TypeID=$p($g(^MR(MRAdm,"DIA",Diag,"TYP",DiagTypeID)),"^",1)
	.i TypeID'="" s TypeDesc=$p($g(^MRC("DTYP",TypeID)),"^",2)
	.q:((Type="ADM")&&(TypeDesc'="入院诊断"))
	.q:((Type="DIS")&&(TypeDesc'="出院诊断"))
	.q:((Type="Main")&&(TypeDesc'="主诊断"))
	.q:((Type="PRE")&&(TypeDesc'="初步诊断"))
	.q:((Type="OP")&&(TypeDesc'="门诊诊断"))
    .s MainDiagFlag=$p($g(^MR(MRAdm,"DIA",Diag,1)),"^",20) //主诊断标志
	.if DiagID'="" Set DiagDesc=$P($g(^MRC("ID",DiagID)),"^",2)
	.else  Set DiagDesc=$g(^MR(MRAdm,"DIA",Diag,"DES",1))
	.;If DiagDesc["-" Set DiagDesc=$P(DiagDesc,"-",2)
	.If $P($g(^MR(MRAdm,"DIA",Diag,1)),"^",20)="Y" s DiagDesc=DiagDesc_"(主)"
	.If DiagStr="" Set DiagStr=DiagDesc
	.Else  Set DiagStr=DiagStr_","_DiagDesc	
	quit DiagStr

In [ ]:
/// @docs:	诊断插入五笔首拼别名, 重复跳过不插入
/// @debug:	d ##class(web.DHCDocMain).AddDiagAlias()
/// @table:
/// 		MRC_ICDDx
/// 		MRC_ICDAlias
ClassMethod AddDiagAlias()
{
	s id = 0
	for {
		s id = $o(^MRC("ID",id))
		q:id=""
		s name = $p(^MRC("ID",id),"^",2)
		s name = ##class(web.DHCINSUPort).GetCNCODE(name,6,"^")
		s new = ""
		for i=1:1:$l(name,"^") {
			s item = $p(name,"^",i)
			s item = $e(item,1,1)
			s new = new _ item	
		}
		s k = 0
		for {
			s k = $o(^MRC("ID",id,"ALIAS",k))
			q:k=""
			s alias = ^MRC("ID",id,"ALIAS",k)
			s:alias=new repeat = 1
		}
		continue:$g(repeat)=1
		s res = ##class(web.DHCBL.CT.MRCICDAlias).SaveAll("^"_new_"^"_id)
	}
}


In [ ]:
/// w ##class(web.DHCDocMain).GetDiagType("161127||6")
ClassMethod GetDiagType(DiagID) [ Language = objectscript ]
{
	q:DiagID="" ""
	s MRAdm = +DiagID
	s Diag = $p(DiagID,"||",2)
	s DiagTypeID=$o(^MR(MRAdm,"DIA",Diag,"TYP",0))
	s TypeID="",TypeDesc=""
	i DiagTypeID'="" s TypeID=$p($g(^MR(MRAdm,"DIA",Diag,"TYP",DiagTypeID)),"^",1)
	i TypeID'="" s TypeDesc=$p($g(^MRC("DTYP",TypeID)),"^",2)
	
	q TypeDesc
}